In [18]:
import warnings
warnings. simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
from tabulate import tabulate
import yfinance as yf
from ta import momentum, trend

In [19]:

def Main_Dataframe(symbol,previes_days):
    # ------------------------------------ DAY TIME FRAME -----------------------------------------------------------
    ticker = yf.Ticker(symbol)
    df = ticker.history(period=f"{previes_days}d", interval="90m")
    df.reset_index(inplace=True)
    df['Date'] = df['Datetime'].dt.date
    df['Time'] = df['Datetime'].dt.time
    
    df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Datetime'], inplace=True)
    df.dropna(inplace=True)
    # -----------ADD EXTRA INDICATORS HERE --------------
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=5).ema_indicator()
    df['15EMA'] = trend.EMAIndicator(df['Close'], window=15).ema_indicator()
    df['Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    df['Prev_5EMA'] = df['5EMA'].shift(1)
    df['Prev_15EMA'] = df['15EMA'].shift(1)

    df['RSI'] = momentum.RSIIndicator(df['Close'], window=6).rsi()
    df['RSI_EMA'] = trend.EMAIndicator(df['RSI'], window=3).ema_indicator()
    
   
    short_ema =trend.EMAIndicator(df["Close"], window=5)
    long_ema = trend.EMAIndicator(df["Close"], window=15)
    
    df["5EMA"] = short_ema.ema_indicator()
    df["15EMA"] = long_ema.ema_indicator()
    
    # Determine the perfect trend based on EMA crossovers
    df["Trend"] = "Sideways"  # Default to Sideways
    df.loc[(df["5EMA"] > df["15EMA"]) & (df["5EMA"].shift(1) <= df["15EMA"].shift(1)), "Trend"] = "Uptrend"
    df.loc[(df["5EMA"] < df["15EMA"]) & (df["5EMA"].shift(1) >= df["15EMA"].shift(1)), "Trend"] = "Downtrend"
    df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

    return df

In [20]:
import pandas as pd
from tabulate import tabulate

class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.trade_logs = []
        self.Capital = 10000
        self.Tred_Day = 2
        self.Charg = 50
        self.Tred_capital = round(self.Capital/self.Tred_Day,2)

    def Get_Lotsize(self,price_per_lot):
        varinace = 100
        quantity_per_lot = 1 
        lot_size = int((self.Tred_capital - self.Charg - varinace)/(quantity_per_lot*price_per_lot))
        return lot_size
        
    def backtesting(self):
        return pd.DataFrame(self.trade_logs)

    def buy(self, equity_name, entry_time, entry_price, qty):
        trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        trade_log['Trade'] = 'Long Open'
        trade_log['Quantity'] = qty
        trade_log['Position Size'] = round(trade_log['Quantity'] * entry_price, 3)
        trade_log['Equity Name'] = equity_name
        trade_log['Entry Time'] = entry_time
        trade_log['Entry Price'] = round(entry_price, 2)
        self.trade_logs.append(trade_log)

    def sell(self, exit_time, exit_price, exit_type, charge):
        trade_log = self.trade_logs[-1]  # Assuming you're selling the most recent buy
        trade_log['Trade'] = 'Long Closed'
        trade_log['Exit Time'] = exit_time
        trade_log['Exit Price'] = round(exit_price, 2)
        trade_log['Exit Type'] = exit_type
        trade_log['PNL'] = round((trade_log['Exit Price'] - trade_log['Entry Price']) * trade_log['Quantity'] - charge, 3)
        trade_log['% PNL'] = round((trade_log['PNL'] / trade_log['Position Size']) * 100, 3)
        trade_log['Holding Period'] = exit_time - trade_log['Entry Time']

    def stats(self):
        df = pd.DataFrame(self.trade_logs)
        parameters = ['Total Trade Scripts', 'Total Trade', 'PNL', 'Winners', 'Losers', 'Win Ratio', 'Total Profit', 'Total Loss', 'Average Loss per Trade', 'Average Profit per Trade', 'Average PNL Per Trade', 'Risk Reward']
        total_traded_scripts = len(df['Equity Name'].unique())
        total_trade = len(df.index)
        pnl = df.PNL.sum()
        winners = len(df[df.PNL > 0])
        losers = len(df[df.PNL <= 0])
        win_ratio = str(round((winners/total_trade) * 100, 2)) + '%'
        total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
        total_loss = round(df[df.PNL <= 0].PNL.sum(), 2)
        average_loss_per_trade = round(total_loss/losers, 2)
        average_profit_per_trade = round(total_profit/winners, 2)
        average_pnl_per_trade = round(pnl/total_trade, 2)
        risk_reward = f'1:{-1 * round(average_profit_per_trade/average_loss_per_trade, 2)}'
        data_points = [total_traded_scripts, total_trade, pnl, winners, losers, win_ratio, total_profit, total_loss, average_loss_per_trade, average_profit_per_trade, average_pnl_per_trade, risk_reward]
        data = list(zip(parameters, data_points))
        print(tabulate(data, ['Parameters', 'Values'], tablefmt='psql'))


In [21]:
def GoldenCrossverSignal(name):
    data = Main_Dataframe(name,50)
    data['Prev_Close'] = data.Close.shift(1)
    data['Signal'] = np.where(data['5EMA'] > data['15EMA'], 1, 0)
    data['Position'] = data.Signal.diff()
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    return df_pos

In [22]:
bt = Backtest()
stock = "CENTRALBK.BO"
data = GoldenCrossverSignal(stock)
required_df = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]
for index, data in required_df.iterrows():
    if(data.Position == 'Buy'):
        qty = bt.Get_Lotsize(data.Close)
        bt.buy(stock, data.DateTime, data.Open, qty)
    else:
        bt.sell(data.DateTime, data.Open, 'Exit Trigger', 0)


In [24]:
bt.stats()
bt.backtesting()
# required_df

+--------------------------+--------------------+
| Parameters               | Values             |
|--------------------------+--------------------|
| Total Trade Scripts      | 1                  |
| Total Trade              | 9                  |
| PNL                      | 389.91999999999996 |
| Winners                  | 5                  |
| Losers                   | 4                  |
| Win Ratio                | 55.56%             |
| Total Profit             | 655.13             |
| Total Loss               | -265.21            |
| Average Loss per Trade   | -66.3              |
| Average Profit per Trade | 131.03             |
| Average PNL Per Trade    | 43.32              |
| Risk Reward              | 1:1.98             |
+--------------------------+--------------------+


,Equity Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,PNL,% PNL,Exit Type,Holding Period
0,CENTRALBK.BO,Long Closed,2023-10-10 15:15:00,49.26,2023-10-11 15:15:00,48.61,98,4827.48,-63.70,-1.320,Exit Trigger,1 days 00:00:00
1,CENTRALBK.BO,Long Closed,2023-10-17 10:45:00,48.68,2023-10-18 10:45:00,48.15,99,4819.32,-52.47,-1.089,Exit Trigger,1 days 00:00:00
2,CENTRALBK.BO,Long Closed,2023-10-27 10:45:00,45.27,2023-10-30 12:15:00,43.90,108,4889.16,-147.96,-3.026,Exit Trigger,3 days 01:30:00
3,CENTRALBK.BO,Long Closed,2023-11-02 10:45:00,44.08,2023-11-09 13:45:00,44.41,110,4848.80,36.30,0.749,Exit Trigger,7 days 03:00:00
4,CENTRALBK.BO,Long Closed,2023-11-13 10:45:00,44.10,2023-11-17 10:45:00,45.95,107,4718.70,197.95,4.195,Exit Trigger,4 days 00:00:00
5,CENTRALBK.BO,Long Closed,2023-11-28 13:45:00,44.42,2023-11-30 09:15:00,44.41,108,4797.36,-1.08,-0.023,Exit Trigger,1 days 19:30:00
6,CENTRALBK.BO,Long Closed,2023-12-01 10:45:00,44.73,2023-12-08 12:15:00,47.84,108,4830.84,335.88,6.953,Exit Trigger,7 days 01:30:00
7,CENTRALBK.BO,Long Closed,2023-12-11 09:15:00,47.25,2023-12-13 10:45:00,47.78,100,4725.00,53.00,1.122,Exit Trigger,2 days 01:30:00
8,CENTRALBK.BO,Long Closed,2023-12-13 13:45:00,47.93,2023-12-15 09:15:00,48.25,100,4793.00,32.00,0.668,Exit Trigger,1 days 19:30:00
